# Data Engineer Phase MVP:

- This notebook serves as the crucial data pipeline for the Defensive Performance Analysis project. Its purpose is to take the raw, uncleaned event data from the provided Excel file, perform all necessary data cleaning and validation checks, and transform the data into a structured format. By creating and mapping pitch zones, this notebook produces a clean, enriched dataset that is then loaded into a PostgreSQL database, ready for the next phases of analysis and visualization.

1. Importing Libraries
- This section is dedicated to importing all the Python libraries required for the data engineering phase. It includes libraries for data manipulation, numerical operations, and database connectivity.

In [250]:
import pandas as pd
import psycopg2


2. Starting extracting all the data from the provided event data excel

In [239]:
raw_excel_data = pd.read_csv("../datasets/Universitatea_Cluj_2024_2025_events.csv")

3. Extract only the useful columns for my mvp and future analysis ideas.

In [ ]:
filtered_raw_excel_data = raw_excel_data[["id","matchId","matchPeriod","minute","second","type.primary","type.secondary","location.x","location.y","team.id","team.name","opponentTeam.id","opponentTeam.name","player.id","player.name","pass.endLocation.x","pass.endLocation.y","shot.xg","shot.isGoal","shot.onTarget","carry.endLocation.x","carry.endLocation.y","possession.id","possession.duration","possession.startLocation.x","possession.startLocation.y","possession.endLocation.x","possession.endLocation.y","competitionId","seasonId","Home_Away"
]]
filtered_raw_excel_data.head(10)

4. Filtering the data so that we can have only the passes and carries vs UCluj

- This step focuses on filtering the raw data to isolate only the opponent's attacking movements against U Cluj. This ensures that our analysis is laser-focused on the defensive problems we're trying to solve.

In [ ]:
new_filtered_raw_excel_data = filtered_raw_excel_data[
    ((filtered_raw_excel_data["type.primary"]=="pass")
    |filtered_raw_excel_data["type.secondary"].apply(lambda x: 'carry' in x))&
    (filtered_raw_excel_data["team.id"]!=60374)]

new_filtered_raw_excel_data["player.name"] = new_filtered_raw_excel_data["player.name"].astype(str)

5. Checking if the columns have unexpected null values
- For the pass and carries end location, they are null where there is no pass or carry event corresponding for that line

In [ ]:
nan_counts_per_column = new_filtered_raw_excel_data.isnull().sum()
print(nan_counts_per_column)

6. Checking if the values for coordinates are accordingly ( between 0-100 )

In [ ]:

is_x_valid = (new_filtered_raw_excel_data["location.x"] >= 0) & (new_filtered_raw_excel_data["location.x"]  <= 100)
is_y_valid = (new_filtered_raw_excel_data["location.y"] >= 0) & (new_filtered_raw_excel_data["location.y"] <= 100)

is_pass_endloc_x_valid = (new_filtered_raw_excel_data["pass.endLocation.x"].notnull() >= 0) & (new_filtered_raw_excel_data["pass.endLocation.x"].notnull() <= 100)
is_pass_endloc_y_valid = (new_filtered_raw_excel_data["pass.endLocation.y"].notnull() >= 0) & (new_filtered_raw_excel_data["pass.endLocation.y"].notnull() <= 100)



all_x_are_valid = is_x_valid.all()
all_y_are_valid = is_y_valid.all()
all_passx_are_valid = is_pass_endloc_x_valid.all()
all_passy_are_valid = is_pass_endloc_y_valid.all()


print(f"All 'location.x' values are valid: {all_x_are_valid}")
print(f"All 'location.y' values are valid: {all_y_are_valid}")
print(f"All 'location.x' values are valid: {all_passx_are_valid}")
print(f"All 'location.y' values are valid: {all_passy_are_valid}")

7. Developing the function for assigning the coordinates in a specific zone from a 16x12 grid size pitch

In [244]:
def calculating_pitch_zone_assignation(x,y):
    row = x//6.25
    column = y//8.33
    result = "zone_"+str(int(row))+"_"+str(int(column))
    return result

8. Applying the zone function for both passes and carries

In [ ]:
# Create a new, explicit copy of the filtered data
df_for_zone_mapping = new_filtered_raw_excel_data.copy()

# Now, apply your calculations to this copy
df_for_zone_mapping["start_zone"] = df_for_zone_mapping.apply(
    lambda row: calculating_pitch_zone_assignation(row["location.x"], row["location.y"]),
    axis=1
)

df_for_zone_mapping["end_zone"] = df_for_zone_mapping.apply(
    lambda row:
        calculating_pitch_zone_assignation(row["pass.endLocation.x"], row["pass.endLocation.y"])
        if not pd.isnull(row["pass.endLocation.x"])
        else calculating_pitch_zone_assignation(row["carry.endLocation.x"], row["carry.endLocation.y"]),
    axis=1
)

df_for_zone_mapping.dtypes

9. Connect to the local PostgreSQL database and execute the SQL queries to create a new schema, create a new table for the cleaned data, and insert the cleaned data into it.

In [ ]:
def read_sql(file_path: str) -> str:
    with open(file_path, "r") as file:
        query = file.read()
        return query
def execute_structural_query(sql: str) -> None:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="placeholder",
        host="localhost",
        port=5432
    )
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    conn.close()
def execute_insert_query(sql: str,data:list) -> None:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="placeholder",
        host="localhost",
        port=5432
    )
    cursor = conn.cursor()
    cursor.execute(read_sql(sql),data)
    conn.commit()
    conn.close()

In [ ]:
if __name__ =="__main__":  
    query_paths_structure = [
        "./SQL_Queries/creating_frf_schema.sql",
        "./SQL_Queries/create_raw_data_table.sql"
        ]
    for query_path in query_paths_structure:
        sql_query = read_sql(query_path)
        execute_structural_query(sql_query)
        #print("done")
    
    for _, row in df_for_zone_mapping.iterrows():
        row_list = row.tolist()
        execute_insert_query("./SQL_Queries/insert_raw_data.sql",row_list)
        #print("done")